This notebook does the following:

    generate random walk polymer chain
    local optimization using openbabel
    packing of multiple polymer chains into the simulation box
    Compute the system topology and read in the force field data
    write the data file for lammps run

Dependencies:

    pymatgen(latest development version from github)
    openbabel with python bindings
    packmol
    rubicon(used only for the packmol interface)

Required input files:

    structures of start, bulk and terminal monomer molecules
    appropriately formatted yaml forcefield data file

In [1]:
import numpy as np

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor

from polymers.core import Polymer
from polymers.utils import get_topology

Set the start, bulk and terminal molecules. Also set the forcefield species name mapping

In [2]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
ff_map = ["Ce", "H", "H", "H", "O", "Cm", "H", "H"]
peo_start.add_site_property("ff_map", ff_map)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
ff_map = ["Ce", "H", "H", "O", "Cm", "H", "H"]
peo_bulk.add_site_property("ff_map", ff_map)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
ff_map = ["Ce", "H", "H", "O", "Cm", "H", "H", "H"]
peo_end.add_site_property("ff_map", ff_map)
e_head = 0
e_tail = 4

Create polymer via random walk

In [3]:
n_units = 10
link_distance = 1.0

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")

Use openbabel to optimize the polymer chain.
Local optimization is used to preserve the chain structure.

In [4]:
peo_obabel = BabelMolAdaptor(peo_polymer.molecule)
# ['gaff', 'ghemical', 'mmff94', 'mmff94s', 'uff']
peo_obabel.localopt(forcefield='mmff94', steps=500)
peo_optimized = peo_obabel.pymatgen_mol

Use Packmol to pack the optimized polymer chains into a box

In [5]:
from rubicon.io.packmol.packmol import PackmolRunner 

# one polymer molecule is fixed at the origin, surrounded by 5 polymer chains within a radius of 200A
packmol_config = [{"number": 1, "fixed": [0, 0, 0, 0, 0, 0],"centerofmass": ""},
                  {"number": 5, "inside sphere": [0, 0, 0, 200.0]}]
pmr = PackmolRunner([peo_optimized, peo_optimized],
                    packmol_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    auto_box=False, output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz


Set the molecular topology and the forcefield paramters. The following needs the latest development version of pymatgen

In [6]:
from pymatgen.io.lammps.topology import Topology
from pymatgen.io.lammps.force_field import ForceField

This is kind of an expensive step, will take couple of minutes for large systems. But this is more thorough and doesnt miss any connectivity info(openbabel misses some)

In [7]:
topology = Topology.from_molecule(packed_polymer)

In [8]:
print len(topology.atoms), len(topology.bonds), len(topology.angles), len(topology.dihedrals)

516 678 1626 2196


Read in the force field parameters

In [9]:
forcefield = ForceField.from_file("ff_data.yaml")

Generate lammps data file from the topology and the forcefield

In [10]:
from pymatgen.io.lammps.data import LammpsForceFieldData

mols = [packed_polymer]
mols_number = [1]
box_size = [[0.0, 300.0], [0.0, 300.0], [0.0, 300.0]]
molecule = packed_polymer
topologies = [topology]
lammps_ff_data = LammpsForceFieldData.from_forcefield_and_topology(mols, mols_number, 
                                                                   box_size, molecule, 
                                                                   forcefield, topologies)

('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('O', 'O') or ('O', 'O') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('O', 'O') or ('O', 'O') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('O', 'O') or ('O', 'O') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('O', 'O') or ('O', 'O') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('H', 'H') or ('H', 'H') Not available
('O', 'O') or ('O', 'O') 

Write the data file(uncomment the write line)

In [11]:
print str(lammps_ff_data)

#write the data file
#lammps_ff_data.write_data_file("lammps_data.dat")

Data file generated by pymatgen

516 atoms
576 bonds
1242 angles
1290 dihedrals

3 atom types
3 bond types
5 angle types
5 dihedral types

0.0 300.0 xlo xhi
0.0 300.0 ylo yhi
0.0 300.0 zlo zhi

Masses 

1 1.00794
2 12.0107
3 15.9994

Bond Coeffs 

1 1000 1.4115
2 1000 1.1041
3 1000 1.5075

Angle Coeffs 

1 42.9 110.1
2 38.5 109.47
3 56.0 109.48
4 86.0 108.54
5 74.5 108.05

Dihedral Coeffs 

1 0.0 0.0 -0.73 0.0
2 0.0 0.0 0.28 0.0
3 1.76 0.67 0.04 0.0
4 0.0 0.0 0.28 0.0
5 0.41 -2.1 -0.6 -0.82

Atoms 

1 1 1 0.0 100.929540148 128.961924926 141.258232465
2 1 1 0.0 100.914080148 127.435824926 140.356582465
3 1 1 0.0 100.914080148 127.435824926 142.159882465
4 1 1 0.0 102.858930148 126.093794926 140.356582465
5 1 1 0.0 102.858930148 126.093794926 142.159882465
6 1 1 0.0 103.611330148 125.574614926 142.245742465
7 1 1 0.0 102.127090148 126.598804926 142.245742465
8 1 1 0.0 103.611320148 125.574614926 144.608692465
9 1 1 0.0 102.127090148 126.598794926 144.608692465
10 1 1 0.0 104.218560148 12